# Searching for hidden genes

Homology-based gene search.

**Input**: Json with missing genes with the format [organism_with_missing_gene, organism_with_present_ortholog, ortholog_ensembl_id], orthology table, ensembl release, path to directory with downloaded genomes, path to directory for target genomic sequences and path to directory for amino acid sequences of orthologs

**Output**: A table with gene search information (orthologs of neighbors, scaffold id and nucleotide sequence, name of BLAST db made from orthologous protein sequences, name of BLAST search result file).

In [3]:
import gget
import pandas as pd
import numpy as np
import requests
import json
import pathlib
import urllib.request
from collections import Counter
from io import StringIO
from Bio import SeqIO

In [4]:
ens_api = "https://rest.ensembl.org"

In [5]:
# enter desired ensembl release
ensembl_release = 109

In [ ]:
# enter path to directory with downloaded genomes
downloaded_genomes_dir = pathlib.Path("genomes/")

In [ ]:
# enter path to directory for saving target region genomic sequences
target_sequences_directory = pathlib.Path("target_sequences/")

In [ ]:
# enter path to directory for saving amino acid sequences of orthologs
ortholog_protein_sequences = pathlib.Path("ortholog_protein_sequences/")

In [ ]:
# enter filename for gene search results
gene_search_result_file = pathlib.Path("100_avian_hidden_genes_search_results")

In [ ]:
# enter JSON with hidden genes and their ortologs
genes_to_search_json_file = pathlib.Path("100_avian_hidden_genes")

In [ ]:
# enter path to orthology table
orthology_table_file = pathlib.Path("/home/vecerkok/orthology_table/orthology_table-68species_ensembl_ids.csv")

## Identify hidden genes

In [7]:
def read_input_json(json_file: str):
    with open(json_file, "r") as f:
        data = json.load(f)
    return data

In [8]:
def get_orthology_table(table_file: str):
    table_df = pd.read_csv(table_file, low_memory=False, index_col=0)
    species_names = table_df.columns
    
    if file_type == ".csv":
        return pd.read_csv(table_file, sep=",", header=0, index_col=0, low_memory=False)
    elif file_type == ".tsv":
        return pd.read_csv(table_file, sep="\t", header=0, index_col=0, low_memory=False)
    elif (file_type == ".xsls") or (file_type == ".xsl"):
        return pd.read_excel(table_file, header=0, index_col=0, low_memory=False)
    else:
        print("Accepted file formats: .csv, .tsv, .xsls, .xsl!")
        return
    
    # modify species names to gtf filename format
    species_names = [x.lower().replace(" ", "_") for x in species_names]

    species_names = list(map(lambda x: x.replace("heterocephalus_glaber", "heterocephalus_glaber_female"), species_names))
    species_names = list(map(lambda x: x.replace("gorilla_gorilla_gorilla", "gorilla_gorilla"), species_names))
    species_names = list(map(lambda x: x.replace("cricetulus_griseus", "cricetulus_griseus_chok1gshd"), species_names))
    species_names = list(map(lambda x: x.replace("ovis_aries", "ovis_aries_rambouillet"), species_names))
    
    table_df.columns = species_names
    return table_df, species_names

## Identify synteny

In [9]:
def is_genome_annotation_downloaded(organism_name: str):
    organism_genome_annotation = downloaded_genomes_dir / (organism_name + ".gtf")
    return organism_genome_annotation.exists()

In [10]:
def download_genome_annotation(organism_name: str):
    # get genome annotation gtf
    gtf_ftp = gget.ref(organism_name, which=["gtf"], release=ensembl_release)[organism_name]["annotation_gtf"]["ftp"]
    print(gtf_ftp)
    
    # download gtf genome nanotation
    genome_file = downloaded_genomes_dir / (organism_name + ".gtf.gz")
    urllib.request.urlretrieve(gtf_ftp, genome_file)
    
    # decompress the genome annotation
    !gzip -d $genome_file

In [11]:
def get_genome_annotation(organism_name: str):
    if is_genome_annotation_downloaded(organism_name):
        print("Genome annotation already downloaded and decompressed!")
        return
    else:
        print("Downloading and then decompressing the genome annotation!")
        download_genome_annotation(organism_name)

In [12]:
def gtf_to_bed(organism_name):
    # convert gtf to bed format
    gtf_file = downloaded_genomes_dir / (organism_name + ".gtf")
    bed_file = downloaded_genomes_dir / ("sorted_" + organism_name + ".bed")
    !gtf2bed < $gtf_file > $bed_file

In [13]:
def find_n_neighbors_in_bed(organism_name: str, ensembl_id, bed_dir, n_neighbors, direction: str):
    bed_df = pd.read_csv(downloaded_genomes_dir / ("sorted_" + organism_name + ".bed"), sep="\t", low_memory=False, usecols=[0, 3], header=None).drop_duplicates(ignore_index=True)
    
    line_index = bed_df.loc[bed_df[3] == ensembl_id].index[0]
    
    if direction == "up":
        if line_index < n_neighbors:
            n_neighbors = line_index
        
        neighbor_indeces = list(range(line_index - n_neighbors, line_index))
        return(bed_df.loc[neighbor_indeces, 3].reset_index(drop=True))
    
    elif direction == "down":
        if line_index + n_neighbors >= len(bed_df):
            n_neighbors = (len(bed_df) - 1) - line_index
        
        line_index += 1
        neighbor_indeces = list(range(line_index, line_index + n_neighbors))
        return(bed_df.loc[neighbor_indeces, 3].reset_index(drop=True))
    
    else:
        print("Wrong direction!")
        return

In [14]:
def find_synteny_neighbors_left(organism_with_ortholog, ortholog_ensembl_id, orthology_table, no_neighbors, genomes_dir):
    gene_row = orthology_table_df.loc[orthology_table_df[organism_with_ortholog] == ortholog_ensembl_id].dropna(axis=1, how="all")
    organisms_with_present = gene_row.columns
    no_organisms_with_present = len(organisms_with_present)
    
    left_neighbors = pd.DataFrame(np.empty(shape=(no_neighbors, no_organisms_with_present)), columns = organisms_with_present)
    left_neighbors[:] = np.nan
    
    for organism in organisms_with_present:
        left_neighbors[organism] = find_n_neighbors_in_bed(organism, gene_row[organism].values[0], genomes_dir, no_neighbors, "up")
        
    return left_neighbors

In [15]:
def find_synteny_neighbors_right(organism_with_ortholog, ortholog_ensembl_id, orthology_table, no_neighbors, genomes_dir):
    gene_row = orthology_table_df.loc[orthology_table_df[organism_with_ortholog] == ortholog_ensembl_id].dropna(axis=1, how="all")
    organisms_with_present = gene_row.columns
    no_organisms_with_present = len(organisms_with_present)
    
    right_neighbors = pd.DataFrame(np.empty(shape=(no_neighbors, no_organisms_with_present)), columns = organisms_with_present)
    right_neighbors[:] = np.nan
    
    for organism in organisms_with_present:
        right_neighbors[organism] = find_n_neighbors_in_bed(organism, gene_row[organism].values[0], genomes_dir, no_neighbors, "down")
        
    return right_neighbors

In [16]:
def find_target_neighbor(neighbors_df, orth_df, organism_with_missing):
    indeces = list()
    for i in range(0, len(neighbors_df)):
        for col in neighbors_df.columns:
            index = orth_df.index[orth_df[col] == neighbors_df.loc[i, col]]
            if len(index) > 0:
                indeces.append(index[0])
    frequencies = Counter(indeces)
    most_common = frequencies.most_common()

    for i in most_common:
        if not pd.isnull(orth_df.loc[i[0], organism_with_missing]):
            return(orth_df.loc[i[0], organism_with_missing])

## Identify target nucleotide sequence

In [17]:
def get_orthologs_json(ensembl_id):
    ext = f"/homology/id/{ensembl_id}"
    response = requests.get(ens_api+ext, headers={"Content-Type" : "application/json"})
    response = response.json()
    return response

In [18]:
def get_neighbor_target_ensembl_id(orthologs, organism_missing):
    if len(orthologs["data"]) > 0:
        for i in range(0,len(orthologs["data"][0]["homologies"])):
            if orthologs["data"][0]["homologies"][i]["target"]["species"] == organism_missing:
                neighbor_target_ensembl_id = orthologs["data"][0]["homologies"][i]["target"]["id"]
                return neighbor_target_ensembl_id

In [19]:
def get_neighbors_in_target_organism(left_neighbor_ortholog, right_neighbor_ortholog, organism_missing):
                
    left_neighbor_orthologs = get_orthologs_json(left_neighbor_ortholog)
    right_neighbor_orthologs = get_orthologs_json(right_neighbor_ortholog)
    
    left_neighbor_target_ensembl_id = get_neighbor_target_ensembl_id(left_neighbor_orthologs, organism_missing)
    right_neighbor_target_ensembl_id = get_neighbor_target_ensembl_id(right_neighbor_orthologs, organism_missing)

    return left_neighbor_target_ensembl_id, right_neighbor_target_ensembl_id

In [20]:
def get_scaffold_id(neigbour_target_ensembl_id):
    ext = f"/lookup/id/{neigbour_target_ensembl_id}?"
    response = requests.get(ens_api+ext, headers={"Content-Type": "application/json"})
    scaffold = response.json()["seq_region_name"]
    
    return scaffold

In [21]:
# TODO get sequence or reverse complement if start > end
def get_coordinates(start, end, scaffold):
    if start < end:
        coordinates = f"{scaffold}:{start}..{end}"
    else:
        coordinates = f"{scaffold}:{end}..{start}"
            
    return coordinates

In [22]:
def get_sequence(organism, coordinates):
    ext = f"/sequence/region/{organism}/{coordinates}?"
    response = requests.get(ens_api+ext, headers={"Content-Type" : "text/x-fasta"})
    target_sequence = response.text
    
    return target_sequence

In [23]:
def get_target_nucleotide_fasta(start, end, scaffold, organism):
    
    # define the region
    coordinates = get_coordinates(start, end, scaffold)
            
    print(f"Retrieving sequence {coordinates}")
    
    # get sequence of target region
    target_sequence = get_sequence(organism, coordinates)
    
    return target_sequence

In [24]:
def get_sequence_from_fasta(fasta: str):
    fasta_io = StringIO(fasta) 

    records = SeqIO.parse(fasta_io, "fasta")
    
    for rec in records:
        sequence = rec.seq

    fasta_io.close()
    
    return str(sequence)

## Identify ortholog protein sequences

In [25]:
def get_missing_gene_orthologs_ensembl_ids(ortholog_id: str, organism_with_ortholog: str, orth_df: pd.DataFrame):
    missing_gene_orthologs_ids = list(orth_df.loc[orth_df[organism_with_ortholog] == ortholog_id, :].values[0])
    missing_gene_orthologs_ids = [x for x in missing_gene_orthologs_ids if type(x) == str]

    return missing_gene_orthologs_ids

In [26]:
def get_list_of_protein_sequences(ensembl_ids):
    protein_sequences_fasta = []
    
    for ens_id in ensembl_ids:
        try:
            sequence = (gget.seq(ens_id, translate=True))
            
            if len(sequence) > 0:
                protein_sequences_fasta.append("\n".join(sequence))
        except TypeError:
            continue
            
    protein_sequences_fasta = [x for x in protein_sequences_fasta if len(x) > 0]

    return protein_sequences_fasta

In [27]:
def save_list_of_sequences_to_fasta(sequences: list, fasta_filename: str):
    with open(fasta_filename, "w") as file:
        for sequence in sequences:
            file.write(sequence)
            file.write("\n")

In [28]:
def make_protein_blast_db_from_fasta(fasta_filename: str):
    !makeblastdb -in $fasta_filename  -parse_seqids -blastdb_version 5 -title $fasta_filename -dbtype prot

In [29]:
def run_blastx(nt_fasta: str, aa_db: str, results_filename: str):
    !blastx -db $aa_db -query $nt_fasta -out $results_filename -outfmt 6

# Analysis

In [ ]:
# load orthology table and list of genes to search
orthology_table_df, species_names = get_orthology_table(orthology_table_file)
genes_to_search = read_input_json(genes_to_search_json_file)

In [30]:
# prepare output dataframe
gene_search_df = pd.DataFrame(columns=["organism_target", "organism_with_ortholog", "ortholog_ensembl_id", "left_neighbor_target", "right_neighbor_target", "left_neighbor_scaffold_id_target", "right_neighbor_scaffold_id_target", "nt_seq_target", "protein_blast_db", "blastx_results"])

In [34]:
# TODO clean up code

for i, trinity in enumerate(genes_to_search):
    try:
        organism_with_missing, organism_with_ortholog, gene_to_search = trinity
        print(f"Searching for gene {gene_to_search}")

        # identify target neighbors
        neighbors_min = 10
        neighbors_max = 15

        left_neighbor_target_ensembl_id = None
        for j in range (neighbors_min, neighbors_max):
            print(j)
            left_neighbors_df = find_synteny_neighbors_left(organism_with_ortholog, gene_to_search, orthology_table_df, j, downloaded_genomes_dir)
            
            left_neighbor_target_ensembl_id = find_target_neighbor(left_neighbors_df, orthology_table_df, organism_with_missing)
            if left_neighbor_target_ensembl_id:
                break

        right_neighbor_target_ensembl_id = None
        for k in range (neighbors_min, neighbors_max):
            print(k)
            right_neighbors_df = find_synteny_neighbors_right(organism_with_ortholog, gene_to_search, orthology_table_df, k, downloaded_genomes_dir)
           
            right_neighbor_target_ensembl_id = find_target_neighbor(right_neighbors_df, orthology_table_df, organism_with_missing)
            if right_neighbor_target_ensembl_id:
                break
                
         # identify scaffolds
        try:
            left_neighbor_scaffold_id = get_scaffold_id(left_neighbor_target_ensembl_id)
        except KeyError:
            print(f"No scaffold id for {left_neighbor_target_ensembl_id}")
            ext = f"/lookup/id/{left_neighbor_target_ensembl_id}?"
            response = requests.get(ens_api+ext, headers={"Content-Type": "application/json"})
            response = response.json()
            print(response)
            left_neighbor_scaffold_id = None

        try:
            right_neighbor_scaffold_id = get_scaffold_id(right_neighbor_target_ensembl_id)
        except KeyError:
            print(f"No scaffold id for {right_neighbor_target_ensembl_id}")
            ext = f"/lookup/id/{right_neighbor_target_ensembl_id}?"
            response = requests.get(ens_api+ext, headers={"Content-Type": "application/json"})
            response = response.json()
            print(response)
            right_neighbor_scaffold_id = None
            
        target_nucleotide_sequence = None
        # retrieve nuclotide sequence from target if on the same scaffold
        if (left_neighbor_scaffold_id == right_neighbor_scaffold_id) and left_neighbor_scaffold_id and right_neighbor_scaffold_id and (left_neighbor_target_ensembl_id != right_neighbor_target_ensembl_id):
            scaffold_id = left_neighbor_scaffold_id

            # identify target sequence coordinates
            left_neighbor_target_end = max(gget.info(left_neighbor_target_ensembl_id, verbose=False)["transcript_ends"].values[0])
            right_neighbor_target_start = min(gget.info(right_neighbor_target_ensembl_id, verbose=False)["transcript_starts"].values[0])

            # get target sequence fasta file and nt sequence in df for futher analysis
            target_nucleotide_fasta = get_target_nucleotide_fasta(left_neighbor_target_end, right_neighbor_target_start, scaffold_id, organism_with_missing)
            
            target_nucleotide_sequence = None
            try:
                target_nucleotide_sequence = get_sequence_from_fasta(target_nucleotide_fasta)
            except Exception as f:
                print(f)

            # write sequence to file
            target_sequence_filename = target_sequences_directory / (organism_with_missing + "_" + gene_to_search + ".fasta")
            with open(target_sequence_filename, "w") as file:
                file.write(target_nucleotide_fasta)

        # write results to table
        
        orthologs_db_filename = None
        blastx_res_filename = None
        if target_nucleotide_sequence:
            orthologs_db_filename = pathlib.Path(organism_with_missing + "_" + gene_to_search + ".fasta")
            orthologs_ids = get_missing_gene_orthologs_ensembl_ids(gene_to_search, organism_with_ortholog, orthology_table_df)
            protein_sequences = get_list_of_protein_sequences(orthologs_ids)
            save_list_of_sequences_to_fasta(protein_sequences, orthologs_db_filename)
            make_protein_blast_db_from_fasta(orthologs_db_filename)
            
            # run blastx
            blastx_res_filename = pathlib.Path(organism_with_missing + "_" + gene_to_search + ".out")
            run_blastx(target_sequence_filename, orthologs_db_filename, blastx_res_filename)
        
        gene_search_df.loc[i, :] = [organism_with_missing, organism_with_ortholog, gene_to_search, left_neighbor_target_ensembl_id, right_neighbor_target_ensembl_id, left_neighbor_scaffold_id, right_neighbor_scaffold_id, target_nucleotide_sequence, orthologs_db_filename, blastx_res_filename]
    except Exception as e:
        print(gene_to_search)
        print(e)

Searching for gene ENSG00000166073
10
10
Retrieving sequence 5:28992581..29114441


Wed Mar 15 15:07:33 2023 INFO Requesting amino acid sequence of the canonical transcript ENST00000561100 of gene ENSG00000166073 from UniProt.
Wed Mar 15 15:07:39 2023 INFO Requesting amino acid sequence of the canonical transcript ENSCGRT00001017466 of gene ENSCGRG00001014423 from UniProt.
Wed Mar 15 15:07:39 2023 WARNING No reviewed UniProt results were found for ID ENSCGRT00001017466. Returning all unreviewed results.
Wed Mar 15 15:07:43 2023 INFO Requesting amino acid sequence of the canonical transcript ENSMAUT00000013146 of gene ENSMAUG00000010507 from UniProt.
Wed Mar 15 15:07:43 2023 WARNING No UniProt sequences were found for ID ENSMAUT00000013146.
Wed Mar 15 15:07:43 2023 ERROR No UniProt amino acid sequences were found for these ID(s).
Wed Mar 15 15:07:50 2023 INFO Requesting amino acid sequence of the canonical transcript ENSRNOT00000007882 of gene ENSRNOG00000005971 from UniProt.
Wed Mar 15 15:07:56 2023 INFO Requesting amino acid sequence of the canonical transcript ENSCP

Wed Mar 15 15:10:42 2023 INFO Requesting amino acid sequence of the canonical transcript ENSLACT00000021192 of gene ENSLACG00000018495 from UniProt.
Wed Mar 15 15:10:43 2023 WARNING No reviewed UniProt results were found for ID ENSLACT00000021192. Returning all unreviewed results.
Wed Mar 15 15:10:50 2023 INFO Requesting amino acid sequence of the canonical transcript ENSLOCT00000015364 of gene ENSLOCG00000012456 from UniProt.
Wed Mar 15 15:10:51 2023 WARNING No reviewed UniProt results were found for ID ENSLOCT00000015364. Returning all unreviewed results.
Wed Mar 15 15:10:55 2023 INFO Requesting amino acid sequence of the canonical transcript ENSDART00000147917 of gene ENSDARG00000042515 from UniProt.
Wed Mar 15 15:10:56 2023 WARNING No UniProt sequences were found for ID ENSDART00000147917.
Wed Mar 15 15:10:56 2023 ERROR No UniProt amino acid sequences were found for these ID(s).
Wed Mar 15 15:11:01 2023 INFO Requesting amino acid sequence of the canonical transcript ENSMOCT00000029

Wed Mar 15 15:13:47 2023 INFO Requesting amino acid sequence of the canonical transcript ENSFCAT00000044218 of gene ENSFCAG00000038334 from UniProt.
Wed Mar 15 15:13:47 2023 WARNING No reviewed UniProt results were found for ID ENSFCAT00000044218. Returning all unreviewed results.
Wed Mar 15 15:13:54 2023 INFO Requesting amino acid sequence of the canonical transcript ENSTTRT00000000034 of gene ENSTTRG00000000034 from UniProt.
Wed Mar 15 15:13:54 2023 WARNING No UniProt sequences were found for ID ENSTTRT00000000034.
Wed Mar 15 15:13:54 2023 ERROR No UniProt amino acid sequences were found for these ID(s).
Wed Mar 15 15:14:00 2023 INFO Requesting amino acid sequence of the canonical transcript ENSLAFT00000015632 of gene ENSLAFG00000015638 from UniProt.
Wed Mar 15 15:14:00 2023 WARNING No reviewed UniProt results were found for ID ENSLAFT00000015632. Returning all unreviewed results.
Wed Mar 15 15:14:08 2023 INFO Requesting amino acid sequence of the canonical transcript ENSECAT00000040



Building a new DB, current time: 03/15/2023 15:14:46
New DB name:   /home/vecerkok/gallus_gallus_ENSG00000166073.fasta
New DB title:  gallus_gallus_ENSG00000166073.fasta
Sequence type: Protein
Deleted existing Protein BLAST database named /home/vecerkok/gallus_gallus_ENSG00000166073.fasta
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 40 sequences in 0.00808191 seconds.




In [35]:
gene_search_df

,organism_target,organism_with_ortholog,ortholog_ensembl_id,left_neighbor_target,right_neighbor_target,left_neighbor_scaffold_id_target,right_neighbor_scaffold_id_target,nt_seq_target,protein_blast_db,blastx_results
0,gallus_gallus,homo_sapiens,ENSG00000166073,ENSGALG00010021084,ENSGALG00010023319,5,5,AAAAGGCTCAGTAGCCAAACCTTTTATTACCCAAAAGTTCTTCAAA...,gallus_gallus_ENSG00000166073.fasta,gallus_gallus_ENSG00000166073.out


In [32]:
gene_search_df.to_csv(gene_search_result_file)